In [ ]:
import rospy

In [ ]:
DEFAULT_I = 10
try:
    i = rospy.get_param('ii')
except:
    print('can not collect param. Going to use default value.')
    i = DEFAULT_I
    
print(i)

In [ ]:
from pcg_gazebo.task_manager import ProcessManager
from pcg_gazebo.generators.creators import create_models_from_config

TARGET_SIM = dict(
    ros_host='localhost',
    ros_port=11311,
    gazebo_host='localhost',
    gazebo_port=11345
)

process_manager = ProcessManager(**TARGET_SIM)
gazebo_proxy = process_manager.get_gazebo_proxy()
print(gazebo_proxy.ros_config)

In [154]:
## spawn box to the any place of wall collision space coordinates

import numpy as np
from pcg_gazebo.generators.shapes import random_rectangle, \
    random_points_to_triangulation, random_rectangles
import trimesh

THRESHOLD_WALL = 0.7
ROOM_SIZE_RADIUS = 4.5
ROOM_WALL_HEIGHT = 0.8
OBSTACLE_NUM = 10
OBSTACLE_SIZE = 0.8
TARGET_SIZE = 0.4

config = [
    dict(
        type='extrude',
        args=dict(
            polygon=random_points_to_triangulation(
                x_min=-ROOM_SIZE_RADIUS,
                x_max=ROOM_SIZE_RADIUS,
                y_min=-ROOM_SIZE_RADIUS,
                y_max=ROOM_SIZE_RADIUS
            ),
            height=ROOM_WALL_HEIGHT,
            thickness=0.1,
            extrude_boundaries=True,
            name='extruded_poly_triangulation',
            color='random'
        )
    )
]

models = create_models_from_config(config)
print(models)
for model in models:
    print(model.get_link_by_name('visual_mesh'))
    print(model.links is None)
    for elem in model.links:
        cols = model.get_link_by_name(elem).collisions
        print(cols[0])
        col_mesh = cols[0]._geometry._geometry_entity.get_meshes()[0]
        sample_pts = cols[0]._geometry._geometry_entity.get_samples(OBSTACLE_NUM)
        print(sample_pts)
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=[0,0,ROOM_WALL_HEIGHT/2]
    )

print(len(col_mesh.vertices))
mini_col_mesh = trimesh.Trimesh(np.array(col_mesh.vertices)*THRESHOLD_WALL)

sample_pts = np.array(mini_col_mesh.convex_hull.sample(len(sample_pts)))
sample_pts += cols[0].pose.position * THRESHOLD_WALL
sample_pts[:,2] = OBSTACLE_SIZE/2

sample_rot = np.zeros([len(sample_pts), 3])
sample_rot[:,2] = np.random.random([len(sample_pts)])*np.pi*2

sizes = [[OBSTACLE_SIZE, OBSTACLE_SIZE, OBSTACLE_SIZE]]*len(sample_pts)

config_obstacle_box = [
    dict(
        type='box_factory',
        args=dict(
            size=sizes,
            name='obstacle_box',
            use_permutation=True,
            color='white'
        )
    )
]

models_box = create_models_from_config(config_obstacle_box)
for model, p, r in zip(models_box, sample_pts, sample_rot):
    print(p, r)
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=list(p),
        rot=list(r)
    )

None
False
<collision name="collision">
  <geometry>
    <mesh>
      <scale>1 1 1</scale>
      <uri>file:///root/.pcg/meshes/uElEWAJcdw.stl</uri>
    </mesh>
  </geometry>
  <max_contacts>20</max_contacts>
  <pose frame="">0.0573531 0.343308 -0.4 0 -0 0</pose>
</collision>

[[ 0.92283347  2.85014762  0.27648333]
 [ 2.26725669 -3.64264483  0.        ]
 [ 4.30851064 -3.36614316  0.53732484]
 [ 0.13536026 -3.3725079   0.58701416]
 [-4.18399713 -3.00383048  0.57225164]
 [ 4.09438226  1.41904182  0.71721679]
 [ 3.05311317 -3.8800475   0.54014222]
 [ 0.92872777 -3.45583203  0.41011709]
 [-4.41348075 -0.50755349  0.28372651]
 [ 0.23529001 -3.38300311  0.58596287]]
190
(array([-0.15119747, -1.48538662,  0.4       ]), array([0.        , 0.        , 3.40741797]))
(array([-0.39131023,  1.43014656,  0.4       ]), array([0.        , 0.        , 4.76632577]))
(array([-0.18375581,  1.53803447,  0.4       ]), array([0.        , 0.        , 5.51429641]))
(array([-1.0545421 ,  0.88052424,  0.4       ]

In [76]:
def get_square_horizon(base_pos, half_length):
    poss = [None]*4
    for i in range(4):
        pos = [0,0,base_pos[2]]
        d1 = [1,1,-1,-1]
        d2 = [1,-1,-1,1]
        pos[0] = base_pos[0] + d1[i]*half_length
        pos[1] = base_pos[1] + d2[i]*half_length
        poss[i] = pos
    
    return poss

In [166]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=np.sqrt(OBSTACLE_SIZE*2), min_samples=1).fit(np.array(sample_pts))
labels = db.labels_
print(labels)

zone_hull = [None]*(max(labels)+1)
for i in range(len(zone_hull)):
    pts = sample_pts[labels==i]
    pts_bottom = np.copy(pts)
    pts_bottom[:,2] = 0
    pts = np.concatenate([pts, pts_bottom])
    print(pts)
    if len(pts) > 4:
#         zone_hull[i] = trimesh.convex.hull_points(pts[:,:2])
        zone_hull[i] = trimesh.Trimesh(vertices=pts).convex_hull
    else:
        v = []
        for p in pts:
            v.extend(get_square_horizon(p, 0.1))
        v = np.array(v)
#         zone_hull[i] = trimesh.convex.hull_points(v[:,:2])
        zone_hull[i] = trimesh.Trimesh(vertices=v).convex_hull
    
print("zone_full: \n{}".format(zone_hull))

zone_samples = [None]*len(zone_hull)
for i in range(len(zone_samples)):
    ## sample_surface return tuple, so choice 0 index
    samples = trimesh.sample.sample_surface(zone_hull[i], 100)[0]
    zone_samples[i] = samples[
        np.logical_and(
            np.not_equal(samples[:,2], 0), 
            np.not_equal(samples[:,2], OBSTACLE_SIZE/2)
        )
    ]

choiced_goal = np.zeros([len(zone_hull),3])
for i in range(len(choiced_goal)):
    choiced_goal[i] = zone_samples[i][np.random.choice(range(len(zone_samples[i])))]
    
choiced_goal[:,2] = TARGET_SIZE/2
print(choiced_goal)

[0 1 1 1 2 0 3 4 2 4]
[[-0.15119747 -1.48538662  0.4       ]
 [-1.27407804 -1.56541337  0.4       ]
 [-0.15119747 -1.48538662  0.        ]
 [-1.27407804 -1.56541337  0.        ]]
[[-0.39131023  1.43014656  0.4       ]
 [-0.18375581  1.53803447  0.4       ]
 [-1.0545421   0.88052424  0.4       ]
 [-0.39131023  1.43014656  0.        ]
 [-0.18375581  1.53803447  0.        ]
 [-1.0545421   0.88052424  0.        ]]
[[ 2.86694563 -1.09823839  0.4       ]
 [ 2.68375094 -0.67498319  0.4       ]
 [ 2.86694563 -1.09823839  0.        ]
 [ 2.68375094 -0.67498319  0.        ]]
[[ 2.39438507 -2.3580727   0.4       ]
 [ 2.39438507 -2.3580727   0.        ]]
[[3.04740332 0.8312627  0.4       ]
 [2.184666   1.42346139 0.4       ]
 [3.04740332 0.8312627  0.        ]
 [2.184666   1.42346139 0.        ]]
zone_full: 
[<trimesh.Trimesh(vertices.shape=(12, 3), faces.shape=(20, 3))>, <trimesh.Trimesh(vertices.shape=(6, 3), faces.shape=(8, 3))>, <trimesh.Trimesh(vertices.shape=(12, 3), faces.shape=(20, 3))>, <t

In [168]:
sizes = [[TARGET_SIZE, TARGET_SIZE, TARGET_SIZE]]*len(choiced_goal)

config_target_box = [
    dict(
        type='box_factory',
        args=dict(
            size=sizes,
            name='target_box',
            use_permutation=True,
            color='cyan'
        )
    )
]

models_target = create_models_from_config(config_target_box)
print(choiced_goal)
for model, p in zip(models_target, choiced_goal):
    print(p)
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=list(p*1.4)
    )

[[-0.06334894 -1.38538662  0.2       ]
 [-0.62046135  1.24024832  0.2       ]
 [ 2.96694563 -1.05256056  0.2       ]
 [ 2.29438507 -2.38823202  0.2       ]
 [ 2.30661982  1.17110822  0.2       ]]
[-0.06334894 -1.38538662  0.2       ]
[-0.62046135  1.24024832  0.2       ]
[ 2.96694563 -1.05256056  0.2       ]
[ 2.29438507 -2.38823202  0.2       ]
[2.30661982 1.17110822 0.2       ]


In [100]:
def delete_models(model_name, start=None, stop=None):
    if start is not None and stop is not None:
        for i in range(start, stop+1):
            gazebo_proxy.delete_model(model_name=model_name+"_"+str(i))
    else:
        gazebo_proxy.delete_model(model_name='')
        
delete_models('box_static_var_size', 0, 10)